それぞれのパスに対応した、AnalysisMemoを作成するための作業用コード。
- 確認されたハニーポットの数(destIpで振り分け)
- ヒット数
- データの更新日

を自動で記載している。

In [ ]:
from DataGrouper import groupBySignature
from DataCollector import importNotDetectData

data = importNotDetectData(2101, 2201)
dataS = groupBySignature(data)
result1 = {}
for s, hits in dataS.items():
    resultIps = {}
    for hit in hits:
        if not hit['_source']['destination_ip'] in resultIps:
            resultIps[hit['_source']['destination_ip']] = []
        resultIps[hit['_source']['destination_ip']].append(hit)
    result1[s] = resultIps
result1 = sorted(result1.items(), key=lambda x: len(dataS[x[0]]), reverse=True)
result1 = sorted(result1, key=lambda x: len(x[1]), reverse=True)

from FormatStandizer import getTimeStrNow

for s, r in result1:
    filename = s
    # replace invalid character
    invalid = '<>:"/\|?* '
    for char in invalid:
        filename = filename.replace(char, '-')
    try:
        f = open(("Memo/" + filename)[:50] + ".md", "x")
    except FileExistsError:
        i = 1
        while True:
            try:
                f = open(("Memo/" + filename)[:50] + "-" + str(i) + ".md", "x")
                break
            except FileExistsError:
                i += 1
    f.write("# Analysis for " + s)
    f.write("\n| POTS | COUNT | CVE | FILE CREATED |")
    f.write("\n|---|---|---|---|")
    f.write("\n| " + str(len(r)) + " | " + str(len(dataS[s])) + " | | " + getTimeStrNow() + " |\n")
    f.write("\n## CVE: ")
    f.close()